In [1]:
import modulos as md
import sys
import os
import constantes as cons
import numpy as np 
import pandas as pd
symbol='STXUSDT'

lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.obtiene_historial(symbol)
        data = md.estrategia_bb(data)
        resultado = md.backtesting(data)
        if resultado[6]>1.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(f"Symbol: {symbol} - Error: {str(falla)} - line: {str(exc_tb.tb_lineno)} - file: {str(fname)}\n")
        pass  


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EOSUSDT - retorno: 6.13% - trades: 6
XLMUSDT - retorno: 1.76% - trades: 5
XTZUSDT - retorno: 2.42% - trades: 7
